In [ ]:
import os
from dotenv import load_dotenv
# from pymilvus import connections

# # If using Docker standalone Milvus
# connections.connect("default", host="127.0.0.1", port="19530")

from pymilvus import connections

load_dotenv(override=True, dotenv_path="../.env.local")

milvus_uri = os.getenv("MILVUS_URI")
milvus_token = os.getenv("MILVUS_API_KEY")


connections.connect(
    alias="default",
    uri=milvus_uri,
    token=milvus_token
)

print("Connected to Milvus on Zilliz Cloud")

In [ ]:
from pymilvus import db
from pymilvus import Collection, FieldSchema, CollectionSchema, DataType

# 1. Create a new database
# db.create_database("rag_db")

# 2. Switch to that database
db.using_database("rag_db")

# ----- Create schema -----
fields = [
    FieldSchema("doc_id", DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema("title", DataType.VARCHAR, max_length=200),
    FieldSchema("domain", DataType.VARCHAR, max_length=100),
    FieldSchema("content", DataType.VARCHAR, max_length=2000),
    FieldSchema("embedding", DataType.FLOAT_VECTOR, dim=384) 
]

schema = CollectionSchema(fields, description="Policy documents with embeddings")
collection = Collection("policy_docs_collection1", schema)

# ----- Create index -----
index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "COSINE",
    "params": {"nlist": 128},
}
collection.create_index(field_name="embedding", index_params=index_params)

In [8]:
# ----- Example data -----
content_chunks = [
    {
        "doc_id": 1,
        "section": "Pay Policies",
        "title": "Employee Pay Policy",
        "domain": "Human Resources",
        "content": "Employees are paid bi-weekly via direct deposit."
    },
    {
        "doc_id": 1,
        "section": "Leave of Absence",
        "title": "Leave Request and Approval Process",
        "domain": "Human Resources",
        "content": "Employees must submit a leave request for approval."
    },
    {
        "doc_id": 1,
        "section": "Internet Use",
        "title": "Acceptable Use of Company Internet",
        "domain": "IT & Security",
        "content": "Company internet must be used for work-related tasks only."
    },
    {
        "doc_id": 2,
        "section": "Break at Work",
        "title": "Employee Break Policy",
        "domain": "Workplace Operations",
        "content": "Employees can take an hour break."
    },
    {
        "doc_id": 2,
        "section": "Harassment",
        "title": "Workplace Harassment Policy",
        "domain": "Compliance",
        "content": "Interact with each employee with respect."
    }
]


# content_chunks_list = []
# for chunk in content_chunks:
#     content_chunks_list.append(chunk["content"])
content_chunks_list = [chunk["content"] for chunk in content_chunks]
print(content_chunks_list)
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

doc_vectors = model.encode(content_chunks_list)
doc_vectors.shape

['Employees are paid bi-weekly via direct deposit.', 'Employees must submit a leave request for approval.', 'Company internet must be used for work-related tasks only.', 'Employees can take an hour break.', 'Interact with each employee with respect.']


c:\Users\sarat\Downloads\GENAI\Training\SLGenerativeAI11126\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(5, 384)

In [ ]:
# ---- Build columnar data ----
doc_ids = [int(i + 1) for i in range(len(content_chunks))]             # INT64
titles = [str(doc["title"]) for doc in content_chunks]                 # VARCHAR
domains = [str(doc["domain"]) for doc in content_chunks]               # VARCHAR
content = [str(doc["content"]) for doc in content_chunks]               # VARCHAR
embeddings = [list(map(float, vec)) for vec in doc_vectors]       # FLOAT_VECTOR(768)


# ---- Insert column-wise ----
collection.insert([doc_ids, titles, domains, content, embeddings])
collection.flush()

print(f"Successfully inserted {len(doc_ids)} documents into Milvus.")

In [5]:
#Load the collection before searching or querying
collection.load()
res = collection.query(expr="doc_id > 0", output_fields=["doc_id", "title", "domain", "content", "embedding"], limit=5)
print(res)

data: ["{'content': 'Employees are paid bi-weekly via direct deposit.', 'embedding': [0.024725107476115227, -0.009081463329494, 0.03887126222252846, 0.032301537692546844, -0.05890548229217529, 0.039124637842178345, 0.026017988100647926, -0.04369091987609863, -0.019160084426403046, -1.12133614038612e-06, 0.013653445057570934, 0.004326742608100176, -0.05378558859229088, 0.006868945900350809, 0.00763819320127368, -0.060696400701999664, -0.03760220855474472, 0.0071409293450415134, 0.13387174904346466, 0.001735346857458353, -0.003075577085837722, -0.07781298458576202, -0.08451439440250397, -0.0032163343857973814, 0.13382798433303833, -0.03643578663468361, 0.016922809183597565, 0.048614807426929474, -0.040287986397743225, 0.019676733762025833, -0.043712910264730453, 0.05151140317320824, -0.016299592331051826, -0.039059169590473175, -0.0044538662768900394, 0.00047448245459236205, -0.031205158680677414, 0.04895620048046112, 0.012831714935600758, 0.01594015397131443, -0.04758511856198311, -0.01

In [ ]:
# print(utility.has_collection("demo_collection"))

# # Get details about a specific collection
# # Get collection details
# collection = Collection("demo_collection")  # instantiate the collection object
print(collection.schema)                    # show the schema
print(collection.num_entities)              # number of entities
print(collection.description)               # optional

In [6]:
# Display results
for record in res:
    print(f"Doc ID: {record['doc_id']}")
    print(f"Title: {record['title']}")
    print(f"Domain: {record['domain']}")
    print(f"Content: {record['content']}")
    # Show only first 5 embedding values for readability
    print(f"Embedding (first 5): {record['embedding'][:5]}")
    print("-" * 80)

Doc ID: 1
Title: Employee Pay Policy
Domain: Human Resources
Content: Employees are paid bi-weekly via direct deposit.
Embedding (first 5): [0.024725107476115227, -0.009081463329494, 0.03887126222252846, 0.032301537692546844, -0.05890548229217529]
--------------------------------------------------------------------------------
Doc ID: 2
Title: Leave Request and Approval Process
Domain: Human Resources
Content: Employees must submit a leave request for approval.
Embedding (first 5): [0.03315500169992447, 0.04853382334113121, 0.04736270383000374, 0.025270450860261917, 0.05931210517883301]
--------------------------------------------------------------------------------
Doc ID: 3
Title: Acceptable Use of Company Internet
Domain: IT & Security
Content: Company internet must be used for work-related tasks only.
Embedding (first 5): [-0.07135910540819168, -0.03066471964120865, 0.03183768689632416, -0.07750098407268524, -0.005032481160014868]
---------------------------------------------------

In [9]:
query = "What's the leave policy?"
query_vector = model.encode([query])[0]
query_vector[:5]  # Show only first 5 values

array([0.06202146, 0.03948993, 0.00912721, 0.0316124 , 0.11862702],
      dtype=float32)

In [10]:
# Search for closest match only in the 'Human Resources' domain
results = collection.search(
    data=[query_vector],
    anns_field="embedding",
    param={"metric_type": "COSINE", "params": {"nprobe": 10}},
    limit=3,
    expr='domain == "Human Resources"',
    output_fields=["doc_id", "title", "domain", "content"]
)

context_sring = ""
for res in results[0]:
    print(f"doc_id={res.entity.get('doc_id')}, "
          f"title={res.entity.get('title')}, "
          f"domain={res.entity.get('domain')}, "
          f"content={res.entity.get('content')}, "
          f"score={res.distance}")
    context_sring += f"\n -- \n {res.entity.get('content')} " # Append content to context string

print("\nContext String for RAG:\n", context_sring)    

doc_id=2, title=Leave Request and Approval Process, domain=Human Resources, content=Employees must submit a leave request for approval., score=0.6372259855270386
doc_id=1, title=Employee Pay Policy, domain=Human Resources, content=Employees are paid bi-weekly via direct deposit., score=0.23553770780563354

Context String for RAG:
 
 -- 
 Employees must submit a leave request for approval. 
 -- 
 Employees are paid bi-weekly via direct deposit. 


In [11]:
from llm_utility import ask_question_open_ai
  
query = "What’s the leave policy?"
response = ask_question_open_ai(query, context_sring)
response

'Based on the provided context, the leave policy is that employees must submit a leave request for approval. No additional details are given.'

In [12]:
print(f"User query: {query}")
print(f"Context: {context_sring}")

print(f"\n\nOpen AI Response: {response}")

User query: What’s the leave policy?
Context: 
 -- 
 Employees must submit a leave request for approval. 
 -- 
 Employees are paid bi-weekly via direct deposit. 


Open AI Response: Based on the provided context, the leave policy is that employees must submit a leave request for approval. No additional details are given.


In [13]:
# RAG Evaluation
# Context_Recall: Did the system retrieve all relevant documents?
# Context_Precision: What proportion of retrieved documents are relevant?

input = "What’s the leave policy?"
expected_context = "Employees are entitled to 20 days of paid leave annually."
actual_context = '''
 Employees must submit a leave request for approval. 
 -- 
 Employees are paid bi-weekly via direct deposit. 
 '''
#Context_Recall: 0%


input = "What’s the leave policy?"
expected_context = ["Employees are entitled to 20 days of paid leave annually."," They must submit a leave request for approval."]
actual_context = ["Employees must submit a leave request for approval. ", " Employees are paid bi-weekly via direct deposit. ", " Company internet must be used for work-related tasks only. "]
#Context_Recall: 1 out of 2 expected = 1/2 = 50%


input = "What’s the leave policy?"
expected_context = ["Employees are entitled to 20 days of paid leave annually."," They must submit a leave request for approval."]
actual_context = ["They are entitled to 20 days of paid leave in a year."," They must submit a leave request for approval."]
#Context_Recall: 2 out of 2 expected = 2/2 = 100%

In [14]:
# Context_Precision: What proportion of retrieved documents are relevant?
input = "What’s the leave policy?"
expected_context = ["Employees are entitled to 20 days of paid leave annually."," They must submit a leave request for approval."]
actual_context = ["Employees must submit a leave request for approval. ", " Employees are paid bi-weekly via direct deposit. ", " Company internet must be used for work-related tasks only. "]
#Context_Precision: 1 out of 3 retrieved = 1/3 = 33%


input = "What’s the leave policy?"
expected_context = ["Employees are entitled to 20 days of paid leave annually."," They must submit a leave request for approval."]
actual_context = ["They are entitled to 20 days of paid leave in a year."," They must submit a leave request for approval.", " Company internet must be used for work-related tasks only. "]
#Context_Precision: 2 out of 3 retrieved = 2/3 = 67%

In [ ]:
##### RAG Evaluation
### Retrieval Metrics
# Context_Recall: Did the system retrieve all relevant documents?
# Context_Precision: What proportion of retrieved documents are relevant?

### Generative Metrics
# Faithfulness
# Accuracy


# F1 = 2 * (Context_Precision * Context_Recall) / (Context_Precision + Context_Recall) -- NOT USED

In [15]:
### Generative Metrics
# Accuracy

input = "What's the leave policy?"
context = ["Employees are entitled to 20 days of paid leave annually."," They must submit a leave request for approval."]

#expected_llm_output = '''
#llm as a judge
groundtruth = ''' 
    Based on the provided context, the leave policy states that employees are entitled to 20 days of paid leave annually 
    and must submit a leave request for approval.'''

actual_llm_output = '''
    Based on the provided context, the leave policy states that employees must submit a leave request for approval. 
    No other details are given. If you have more of the policy, I can summarize that as well.'''

accuracy = 0.50


# Faithfulness - Is the generated output consistent with the provided context?
faithfulness = 0.50